In [4]:
import matplotlib as mpl
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow.compat.v1 as tf
from sklearn.preprocessing import LabelEncoder
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
import time
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import roc_auc_score
import scipy.stats as st
from sympy.solvers import solve
from sympy import Symbol
from scipy.optimize import fsolve
from sklearn.metrics import fbeta_score,f1_score,balanced_accuracy_score,recall_score,precision_score
from sklearn.neural_network import MLPClassifier
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from cvxopt import matrix
from cvxopt import solvers
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Flatten, Activation, GlobalAveragePooling2D,Conv2D, MaxPooling2D
from sklearn.ensemble import AdaBoostClassifier # AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier # KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier 
from tensorflow.keras.models import Model, Sequential
import math

# Data preparation

In [2]:
ecoli=pd.read_csv('/Users/lvjingzhe/Desktop/璇/modified_althogram/DRM/data/ecoli_.csv',)

In [3]:
ecoli

,Mcg,Gvh,Lip,Chg,Aac,Alm1,Alm2,Class
0,49,29,48,5,56,24,35,0
1,7,4,48,5,54,35,44,0
2,56,4,48,5,49,37,46,0
3,59,49,48,5,52,45,36,0
4,23,32,48,5,55,25,35,0
...,...,...,...,...,...,...,...,...
331,74,56,48,5,47,68,3,2
332,71,57,48,5,48,35,32,2
333,61,6,48,5,44,39,38,2
334,59,61,48,5,42,42,37,2


In [5]:
L=[]
for k in ecoli['Class'].values:
    if k in ['0','1','2','3' ]:
        L.append(0)
    else:L.append(1)
ecoli['Class']=L       

In [6]:
ecoli['Class'].value_counts()

0    307
1     29
Name: Class, dtype: int64

In [7]:
E=np.array(ecoli)
np.random.seed(4123)
np.random.shuffle(E)
X_sample=E[:,:-1]/np.max(E[:,:-1],axis=0)
y_sample=E[:,-1]
X_sample.shape

(336, 7)

In [8]:
x_tr, x_te, y_tr, y_te = train_test_split(X_sample,y_sample,test_size = 0.2,
                                                  shuffle = True,
                                                  random_state = 4123)

In [9]:
X_train=np.c_[x_tr,np.ones(x_tr.shape[0])]
X_test=np.c_[x_te,np.ones(x_te.shape[0])]

In [10]:
cv_= RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=0)
cv=RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=0)


In [11]:
import time
start_i=time.time()
model_in=LogisticRegression()
model_in.fit(x_tr,y_tr)
# model_in.fit(X_resampled_smote,y_resampled_smote)
end_i=time.time()
y_train_proba=model_in.predict_proba(x_tr)
y_train_label=model_in.predict(x_tr)
y_test_proba=model_in.predict_proba(x_te)

y_test_label=model_in.predict(x_te)

from sklearn.metrics import classification_report
print(classification_report(y_te,y_test_label))
print("Running time:%.2f s"%(end_i-start_i))
from sklearn.metrics import accuracy_score

print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2)))
print('Accuracy on training set：{:.2%}'.format(accuracy_score(y_tr,y_train_label)))
print('Accuracy on testing set：{:.2%}'.format(accuracy_score(y_te,y_test_label)))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97        63
           1       1.00      0.20      0.33         5

    accuracy                           0.94        68
   macro avg       0.97      0.60      0.65        68
weighted avg       0.94      0.94      0.92        68

Running time:0.01 s
Balanced-Accuracy on testing set：60.00%
Recall on testing set：20.00%
F-measure on testing set：23.81%
Accuracy on training set：93.66%
Accuracy on testing set：94.12%


## DRM

In [12]:
'''Order the dataset by label'''
X_sample_ord=np.concatenate((X_sample[y_sample==0],X_sample[y_sample==1]))
y_sample_ord=np.concatenate((y_sample[y_sample==0],y_sample[y_sample==1]))

In [13]:
'''Algorithm for DRM'''


class Poly_close_solution:
    def __init__(self,kernel='self._poly',penalty = None,gamma=1,degree = 2,coef0=0,alpha=1,beta=1):
        self.W = None
        self.penalty = penalty
        self.d = degree
        self.b = coef0
        self.alpha=alpha
        self.beta=beta
        self.g=gamma
        self.kernel=kernel
        
    def _poly(self,X_1,X_2):
        return (self.g*(X_1.dot(X_2.T))+self.b)**self.d
    def _rbf(self,x,y):
        return np.exp(-self.g*np.sum((x[...,None,:]-y)**2,axis=2))

    def poly_B_matrix(self,X,Y):
        n=Y[Y==1].shape[0]
        X_sample_split=np.array(np.split(X,np.array([X.shape[0]-n])))
        B=np.zeros((X.shape[0],X.shape[0]))
        I=0
        for  m in X_sample_split:
            I+=m.shape[0]
            B[I-m.shape[0]:I,I-m.shape[0]:I]=eval(self.kernel+'(m,m)')/m.shape[0]
        return B
    
    def K_x(self,X,x_t):
        return eval(self.kernel+'(X,x_t)')
    
    def QplusBeta(self,X,Y):
        s=time.time()
        K=eval(self.kernel+'(X,X)')
        H_p=np.diag(np.diagonal(K))
        B_p=self.poly_B_matrix(X,Y)
        Qplus_beta=K+self.alpha*(H_p-B_p)+np.diag([self.beta]*X.shape[0])
        e=time.time()
        return K,Qplus_beta
    def fit(self,invQ,X,x_t):
        
        
        W=invQ.dot(self.K_x(X,x_t))##求逆耗时
        
        return W.reshape(-1)
    def delta_phi(self,K,invQ,X,Y,x_t,j):
        
        w=self.fit(invQ,X,x_t)
        w_noty=w

        w_y=np.zeros((X.shape[0]))
        indice=np.argwhere(Y==j).reshape(-1)
        w_noty[indice]=0
        w_y[indice]=w[indice]
        delta=np.dot(w_y,K.dot(w_y))+np.dot(w_noty,K.dot(w_noty))-2*np.dot(w_y,self.K_x(X,x_t))
        return delta
    def predict(self,K,invQ,X,Y,x_t):
        delta=[]
        for k in np.unique(Y):
            delta.append(self.delta_phi(K,invQ,X,Y,x_t,k))
        return np.argmin(delta)

In [14]:
from numpy.linalg import inv,det

In [15]:
'''Parameter selection for DRM'''


from sklearn.metrics import fbeta_score
from numpy.linalg import inv,det
metrics_names=['G-mean','Balanced_acc','F1_score','Recall','Precision']
def DRM_FiveCV(X,y,model=Poly_close_solution(
                                )):
    
    G_m,bacc,f2,rec,pre=list(),list(),list(),list(),list()
    for train_index,test_index in cv_.split(X,y):
                
                K,Q_beta=model.QplusBeta(X[train_index],y[train_index])
                invQ=inv(Q_beta)
                y_test_label=list()
                for i in range(y[test_index].shape[0]):
                    y_test_label.append(model.predict(K,invQ,X[train_index],y[train_index],X[test_index][i]))

                rec.append(recall_score(y[test_index],y_test_label))
                pre.append(precision_score(y[test_index],y_test_label))
                G_m.append(math.sqrt(recall_score(y[test_index],y_test_label)*recall_score(y[test_index],y_test_label,pos_label=0)))
                bacc.append(balanced_accuracy_score(y[test_index],y_test_label))
                f2.append(fbeta_score(y[test_index],y_test_label,beta=2))
 
    return G_m,bacc,f2,rec,pre


In [16]:
'''Parameter(dgree,alpha,beta) selection for DRM'''

degree=np.array([2,3,4,5,8,10])##tr:te=8:2
gamma=np.array([0.001,0.01,0.1,1,10])
parameter=np.array([0.01,0.1,1,10,100])
X_tr_ord=np.concatenate((x_tr[y_tr==0],x_tr[y_tr==1]))
y_tr_ord=np.concatenate((y_tr[y_tr==0],y_tr[y_tr==1]))
all_results_Ldrm = []
import time
import math
# for k in mean_lr:
start=time.time()

for k in parameter:
    for p in parameter:
        for d in degree:
            Lo_score=np.mean(DRM_FiveCV(X_tr_ord,y_tr_ord,model=Poly_close_solution(alpha=k,beta=p,degree=d
                                )),axis=1)

            metric_res = {'alpha':k,'beta':p,'degree':d}

            for name, value in zip(metrics_names, Lo_score):
        #             print(name, ': ', value)
                    metric_res[name] = value


            all_results_Ldrm.append(metric_res)
end=time.time()
print("DRM_L_fivefold-cv time:%.2f"%(end-start))

DRM_L_fivefold-cv time:20.30


In [74]:
pd.DataFrame(all_results_Ldrm).to_csv('/Users/lvjingzhe/Desktop/璇/modified_althogram/code/0511CV_select_DRML_keelecoli4567.csv')


In [17]:
degree=np.array([2,3,4,5,8,10])
gamma=np.array([0.001,0.01,0.1,1,10])
parameter=np.array([0.01,0.1,1,10,100])
def fivetrials_DRM(X,y):
    
    
    G_m,bacc,fbeta,rec,pre,T=list(),list(),list(),list(),list(),list()
    
    for train_index,test_index in cv.split(X, y):

        start=time.time()
        P_model=Poly_close_solution(alpha=1,beta=0.01,degree=10)
        K,Q_beta=P_model.QplusBeta(X[train_index],y[train_index])
        invQ=inv(Q_beta)
        end=time.time()
        prdict_y=[]
        for i in range(y[test_index].shape[0]):
            prdict_y.append(P_model.predict(K,invQ,X[train_index],y[train_index],X[test_index][i]))
        
        
        print(classification_report(y[test_index],prdict_y))
        rec.append(recall_score(y[test_index],prdict_y))
        G_m.append(np.sqrt(recall_score(y[test_index],prdict_y)*recall_score(y[test_index],prdict_y,pos_label=0)))
        pre.append(precision_score(y[test_index],prdict_y))
        bacc.append(balanced_accuracy_score(y[test_index],prdict_y))
        fbeta.append(fbeta_score(y[test_index],prdict_y,beta=max(2,np.log(y_sample.shape[0]/sum(y_sample)-1))))
        T.append(end-start)




    return G_m,bacc,fbeta,rec,pre,T

In [54]:
result_DRM = fivetrials_DRM(X_sample_ord,y_sample_ord)

G_mean=result_DRM[0]
Bacc=result_DRM[1]
# summarize performance
recall=result_DRM[3]
Fbeta=result_DRM[2]
T=result_DRM[-1]
acc_s=np.mean(np.array(result_DRM)[:4,:],axis=0)
	# summarize and store
print('>%s: Average G-mean:%.3f(%.3f) ' % ('DRM:',np.mean(G_mean),np.std(G_mean)))
print('>%s: Average Balanced_Acc: %.3f(%.3f) ' % ('DRM:',np.mean(Bacc),np.std(Bacc)))
print('>%s: Average Fbeta: %.3f(%.3f)' % ('DRM:',np.mean(Fbeta),np.std(Fbeta)))
print('>%s: Average Recall: %.3f(%.3f)' % ('DRM:',np.mean(recall),np.std(recall)))    
print('>%s: Average Training time: %.3f(%.3f)' % ('DRM:',np.mean(T),np.std(T)))
print('>%s: Average accuracy_score: %.3f(%.3f)' % ('DRM:',np.mean(acc_s),np.std(acc_s)))
print('>%s: Average Score: %.3f(%.3f)' % ('DRM:',np.mean(0.99*acc_s+0.01/(np.array(result_DRM)[-1,:]/10*9+1)),np.std(0.99*acc_s+0.01/(np.array(result_DRM)[-1,:]/10*9+1))))
          

              precision    recall  f1-score   support

           0       0.95      0.98      0.97        62
           1       0.75      0.50      0.60         6

    accuracy                           0.94        68
   macro avg       0.85      0.74      0.78        68
weighted avg       0.94      0.94      0.94        68

              precision    recall  f1-score   support

           0       0.97      1.00      0.98        62
           1       1.00      0.60      0.75         5

    accuracy                           0.97        67
   macro avg       0.98      0.80      0.87        67
weighted avg       0.97      0.97      0.97        67

              precision    recall  f1-score   support

           0       0.98      0.98      0.98        61
           1       0.83      0.83      0.83         6

    accuracy                           0.97        67
   macro avg       0.91      0.91      0.91        67
weighted avg       0.97      0.97      0.97        67

              preci

## SOTA methods：LR、SVM、AUSTBOOST、BAGGING...+Sampling

In [22]:
'''Alternative baseline models, with parameters selected'''


from sklearn import tree
def get_models():
	models, names = list(), list()
	# LR
	models.append(LogisticRegression())
	names.append('LR')
	# SVM
	models.append(SVC(probability=True))
	names.append('SVM')
	# Bagging
	models.append(BaggingClassifier(n_estimators=1000))
	names.append('BAG')
	models.append(AdaBoostClassifier(random_state=0))
	names.append('Adaboost')
	# RF
	models.append(
	tree.DecisionTreeClassifier(max_leaf_nodes=6,
                               random_state=0))
	names.append('CART')
	models.append(RandomForestClassifier(criterion='entropy',n_estimators=6))
	names.append('RF')
	# GBM
	models.append(MLPClassifier(random_state=0, max_iter=200,hidden_layer_sizes=(50,)))
    
	names.append('MLP')

	return models, names

In [23]:

'''Performances of baseline+sampling methods on five trials'''

import time
def fivetrials(X,y,model,sample='none'):
    pipeline = model
    
    G_m,bacc,fbeta,rec,pre,T=list(),list(),list(),list(),list(),list()
    if sample!='none':
        for train_index,test_index in cv.split(X, y):
        
            start=time.time()
            resample_x,resample_y=sample.fit_resample(X[train_index],y[train_index])
            
            pipeline.fit(resample_x,resample_y)
            end=time.time()
            prdict_y=pipeline.predict(X[test_index])
            prob_y=pipeline.predict_proba(X[test_index])
            rec.append(recall_score(y[test_index],prdict_y))
            G_m.append(np.sqrt(recall_score(y[test_index],prdict_y)*recall_score(y[test_index],prdict_y,pos_label=0)))
            pre.append(precision_score(y[test_index],prdict_y))
            bacc.append(balanced_accuracy_score(y[test_index],prdict_y))
            fbeta.append(fbeta_score(y[test_index],prdict_y,beta=max(2,np.log(y_sample.shape[0]/sum(y_sample)-1))))
            T.append(end-start)
    else:
        for train_index,test_index in cv.split(X, y):
    
            start=time.time()
            pipeline.fit(X[train_index],y[train_index])
            end=time.time()
            prdict_y=pipeline.predict(X[test_index])
            prob_y=pipeline.predict_proba(X[test_index])
            rec.append(recall_score(y[test_index],prdict_y))
            G_m.append(np.sqrt(recall_score(y[test_index],prdict_y)*recall_score(y[test_index],prdict_y,pos_label=0)))
            pre.append(precision_score(y[test_index],prdict_y))
            bacc.append(balanced_accuracy_score(y[test_index],prdict_y))
            fbeta.append(fbeta_score(y[test_index],prdict_y,beta=max(2,np.log(y_sample.shape[0]/sum(y_sample)-1))))
            T.append(end-start)




    return G_m,bacc,fbeta,rec,pre,T


In [24]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
X_resampled,y_resampled=RandomUnderSampler().fit_resample(x_tr,y_tr)
X_resampled_smote,y_resampled_smote  = SMOTE().fit_resample(x_tr, y_tr)

In [56]:
models, names = get_models()

for i in range(len(models)):
	result = fivetrials(X_sample,y_sample,models[i])

	G_mean=result[0]
	Bacc=result[1]

	recall=result[3]
	Fbeta=result[2]
	T=result[-1]
	acc_s=np.mean(np.array(result)[:4,:],axis=0)
	print('>%s: Average G-mean:%.3f(%.3f) ' % (names[i],np.mean(G_mean),np.std(G_mean)))
	print('>%s: Average Balanced_Acc: %.3f(%.3f) ' % (names[i],np.mean(Bacc),np.std(Bacc)))
	print('>%s: Average Fbeta: %.3f(%.3f)' % (names[i],np.mean(Fbeta),np.std(Fbeta)))
	print('>%s: Average Recall: %.3f(%.3f)' % (names[i],np.mean(recall),np.std(recall)))    
	print('>%s: Average Training time: %.3f(%.3f)' % (names[i],np.mean(T),np.std(T)))
	print('>%s: Average accuracy_score: %.3f(%.3f)' % (names[i],np.mean(acc_s),np.std(acc_s)))
	print('>%s: Average Score: %.3f(%.3f)' % (names[i],np.mean(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1)),np.std(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1))))
          

>LR: Average G-mean:0.334(0.168) 
>LR: Average Balanced_Acc: 0.570(0.036) 
>LR: Average Fbeta: 0.160(0.081)
>LR: Average Recall: 0.140(0.071)
>LR: Average Training time: 0.017(0.022)
>LR: Average accuracy_score: 0.301(0.089)
>LR: Average Score: 0.308(0.088)
>SVM: Average G-mean:0.804(0.065) 
>SVM: Average Balanced_Acc: 0.825(0.052) 
>SVM: Average Fbeta: 0.684(0.095)
>SVM: Average Recall: 0.653(0.109)
>SVM: Average Training time: 0.015(0.011)
>SVM: Average accuracy_score: 0.741(0.080)
>SVM: Average Score: 0.744(0.079)
>BAG: Average G-mean:0.830(0.105) 
>BAG: Average Balanced_Acc: 0.848(0.081) 
>BAG: Average Fbeta: 0.721(0.151)
>BAG: Average Recall: 0.713(0.169)
>BAG: Average Training time: 1.483(0.138)
>BAG: Average accuracy_score: 0.778(0.127)
>BAG: Average Score: 0.775(0.125)
>Adaboost: Average G-mean:0.736(0.158) 
>Adaboost: Average Balanced_Acc: 0.782(0.105) 
>Adaboost: Average Fbeta: 0.595(0.208)
>Adaboost: Average Recall: 0.573(0.214)
>Adaboost: Average Training time: 0.063(0.001)

In [26]:
models, names = get_models()

for i in range(len(models)):
	result = fivetrials(X_sample,y_sample,models[i],sample=SMOTE(random_state=0))

	G_mean=result[0]
	Bacc=result[1]
	recall=result[3]
	Fbeta=result[2]
	T=result[-1]
	acc_s=np.mean(np.array(result)[:4,:],axis=0)
	print('>%s: Average G-mean:%.3f(%.3f) ' % ('SMOTE'+names[i],np.mean(G_mean),np.std(G_mean)))
	print('>%s: Average Balanced_Acc: %.3f(%.3f) ' % ('SMOTE'+names[i],np.mean(Bacc),np.std(Bacc)))
	print('>%s: Average Fbeta: %.3f(%.3f)' % ('SMOTE'+names[i],np.mean(Fbeta),np.std(Fbeta)))
	print('>%s: Average Recall: %.3f(%.3f)' % ('SMOTE'+names[i],np.mean(recall),np.std(recall)))    
	print('>%s: Average Training time: %.3f(%.3f)' % ('SMOTE'+names[i],np.mean(T),np.std(T)))
	print('>%s: Average accuracy_score: %.3f(%.3f)' % ('SMOTE'+names[i],np.mean(acc_s),np.std(acc_s)))
	print('>%s: Average Score: %.3f(%.3f)' % ('SMOTE'+names[i],np.mean(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1)),np.std(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1))))
          

>SMOTELR: Average G-mean:0.888(0.074) 
>SMOTELR: Average Balanced_Acc: 0.891(0.071) 
>SMOTELR: Average Fbeta: 0.779(0.125)
>SMOTELR: Average Recall: 0.860(0.127)
>SMOTELR: Average Training time: 0.007(0.003)
>SMOTELR: Average accuracy_score: 0.854(0.098)
>SMOTELR: Average Score: 0.856(0.097)
>SMOTESVM: Average G-mean:0.832(0.081) 
>SMOTESVM: Average Balanced_Acc: 0.847(0.066) 
>SMOTESVM: Average Fbeta: 0.718(0.119)
>SMOTESVM: Average Recall: 0.720(0.142)
>SMOTESVM: Average Training time: 0.016(0.001)
>SMOTESVM: Average accuracy_score: 0.779(0.102)
>SMOTESVM: Average Score: 0.781(0.101)
>SMOTEBAG: Average G-mean:0.850(0.082) 
>SMOTEBAG: Average Balanced_Acc: 0.860(0.074) 
>SMOTEBAG: Average Fbeta: 0.741(0.131)
>SMOTEBAG: Average Recall: 0.753(0.145)
>SMOTEBAG: Average Training time: 1.654(0.042)
>SMOTEBAG: Average accuracy_score: 0.801(0.108)
>SMOTEBAG: Average Score: 0.797(0.107)
>SMOTEAdaboost: Average G-mean:0.800(0.115) 
>SMOTEAdaboost: Average Balanced_Acc: 0.822(0.090) 
>SMOTEAdab

In [27]:
models, names = get_models()

for i in range(len(models)):
	result = fivetrials(X_sample,y_sample,models[i],sample=RandomUnderSampler(random_state=42))

	G_mean=result[0]
	Bacc=result[1]
	recall=result[3]
	Fbeta=result[2]
	T=result[-1]
	acc_s=np.mean(np.array(result)[:4,:],axis=0)
	print('>%s: Average G-mean:%.3f(%.3f) ' % ('RUS'+names[i],np.mean(G_mean),np.std(G_mean)))
	print('>%s: Average Balanced_Acc: %.3f(%.3f) ' % ('RUS'+names[i],np.mean(Bacc),np.std(Bacc)))
	print('>%s: Average Fbeta: %.3f(%.3f)' % ('RUS'+names[i],np.mean(Fbeta),np.std(Fbeta)))
	print('>%s: Average Recall: %.3f(%.3f)' % ('RUS'+names[i],np.mean(recall),np.std(recall)))    
	print('>%s: Average Training time: %.3f(%.3f)' % ('RUS'+names[i],np.mean(T),np.std(T)))
	print('>%s: Average accuracy_score: %.3f(%.3f)' % ('RUS'+names[i],np.mean(acc_s),np.std(acc_s)))
	print('>%s: Average Score: %.3f(%.3f)' % ('RUS'+names[i],np.mean(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1)),np.std(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1))))
          


>RUSLR: Average G-mean:0.862(0.055) 
>RUSLR: Average Balanced_Acc: 0.866(0.052) 
>RUSLR: Average Fbeta: 0.726(0.087)
>RUSLR: Average Recall: 0.860(0.127)
>RUSLR: Average Training time: 0.005(0.001)
>RUSLR: Average accuracy_score: 0.828(0.077)
>RUSLR: Average Score: 0.830(0.076)
>RUSSVM: Average G-mean:0.878(0.048) 
>RUSSVM: Average Balanced_Acc: 0.883(0.045) 
>RUSSVM: Average Fbeta: 0.758(0.066)
>RUSSVM: Average Recall: 0.860(0.127)
>RUSSVM: Average Training time: 0.002(0.000)
>RUSSVM: Average accuracy_score: 0.845(0.071)
>RUSSVM: Average Score: 0.846(0.070)
>RUSBAG: Average G-mean:0.845(0.059) 
>RUSBAG: Average Balanced_Acc: 0.848(0.058) 
>RUSBAG: Average Fbeta: 0.691(0.078)
>RUSBAG: Average Recall: 0.867(0.125)
>RUSBAG: Average Training time: 1.214(0.027)
>RUSBAG: Average accuracy_score: 0.813(0.079)
>RUSBAG: Average Score: 0.809(0.078)
>RUSAdaboost: Average G-mean:0.794(0.054) 
>RUSAdaboost: Average Balanced_Acc: 0.798(0.054) 
>RUSAdaboost: Average Fbeta: 0.613(0.084)
>RUSAdaboost: 

#  Max-mean loss method
" Q. Xu, X. M. Xuan, Nonlinear regression without iid assumption"

In [59]:
X_sample_XX=np.c_[X_sample,np.ones(X_sample.shape[0])]

In [46]:
'''Split the original data into m groups, one of which contains the positive class Data'''

def split(x,y):
    k=int(y.shape[0]/sum(y))-1
    n=y[y==1].shape[0]
    m=int((x.shape[0]-n)/k)
    indice=[i*m for i in range(1,k)]
    indice.append(x.shape[0]-n)
    x_c=np.concatenate((x[y==0],x[y==1]))
    y_c=np.concatenate((np.array([0]*(x.shape[0]-n)),np.array([1]*n)))
    return np.array(np.split(x_c,indice)),np.array(np.split(y_c,indice))




In [28]:
'''Althogrim for max-mean loss regression'''

class solution_XX:
    
    
    def __init__(self,penalty = None,Lambda = 0.03,a = 0.5,epochs = 200):
        self.W = None
        self.penalty = penalty
        self.Lambda = Lambda
        self.a = a
        self.epochs =epochs
        self.sigmoid = lambda x:1/(1 + np.exp(-x))


    def f_XX(self,X,Y):
        if self.penalty=='l1':f=np.array([np.mean((self.sigmoid(np.dot(x,self.W))-y)**2 )+self.Lambda*np.sum(np.abs(self.W)) for x,y in zip(X,Y)])
        elif self.penalty=='l2':f=np.array([np.mean((self.sigmoid(np.dot(x,self.W))-y)**2 )+self.Lambda*np.sum(self.W**2) for x,y in zip(X,Y)])#pre_Xtrain,pre_Ytrain
        else:f=np.array([np.mean((self.sigmoid(np.dot(x,self.W))-y)**2 ) for x,y in zip(X,Y)])

        return f         
    def Gf_XX(self,X,Y):#To compute the Derivative matrix, the shape of which is N*2
        if self.penalty=='l1':d=np.array([x.T.dot(0.02*(self.sigmoid(np.dot(x,self.W))-y)*self.sigmoid(np.dot(x,self.W))*(1-self.sigmoid(np.dot(x,self.W))))+self.Lambda*np.sign(self.W )for x,y in zip(X,Y)])
        elif self.penalty=='l2':
            d=np.array([x.T.dot(0.02*(self.sigmoid(np.dot(x,self.W))-y)*self.sigmoid(np.dot(x,self.W))*(1-self.sigmoid(np.dot(x,self.W))))+2*self.Lambda*self.W for x,y in zip(X,Y)])

        else:d=np.array([x.T.dot(0.02*(self.sigmoid(np.dot(x,self.W))-y)*self.sigmoid(np.dot(x,self.W))*(1-self.sigmoid(np.dot(x,self.W)))) for x,y in zip(X,Y)])
        return d
    def direction_XX(self,X,Y):
        gra=self.Gf_XX(X,Y)
        p=matrix(gra.dot(gra.T),tc='d')
        q=matrix(-self.f_XX(X,Y),tc='d')
        G=matrix(np.diag(np.array([-1]*Y.shape[0])),tc='d')#N=20
        h=matrix(np.array([[0]]*Y.shape[0]),tc='d')
        A=matrix([[1.0]]*Y.shape[0])
        b=matrix([1.0])
        solvers.options['show_progress'] = False
        sol = solvers.qp(p,q,G,h,A,b)
        t=np.array(sol['x'])
        d= -(gra.T.dot(t))
        return d.reshape((X_train.shape[-1],))

    def fit(self,X,Y):
        
        call=[]
        pre=[]
        loss=[]
        testloss=[]
        np.random.seed(1324)
        self.W=np.random.random((X_train.shape[-1],))*2-1
        n=y_te[y_te==1.].shape[0]
        for k in range(200):
            d=self.direction_XX(X,Y)
            if np.linalg.norm(d)//10**(-7) < 25:
                break
            sigma=0.8
            f_1=np.max(self.f_XX(X,Y))
            w=self.W
            self.W=d*sigma+w
            while np.max(self.f_XX(X,Y))>np.max(f_1):
                sigma=sigma*0.8
                self.W=d*sigma+w
            self.W=d*sigma+w


        return self.W

In [29]:
'''Performances of max-mean loss regression on five trials'''
def fivetrials_XX(X,y,model=solution_XX()):
    pipeline = model
    
    G_m,bacc,fbeta,rec,pre,T=list(),list(),list(),list(),list(),list()
    
    for train_index,test_index in cv.split(X, y):
        
        start=time.time()

        pre_X,pre_Y=split(X[train_index],y[train_index])

        w_x=pipeline.fit(pre_X,pre_Y)
        end=time.time()
        prdict_y=pipeline.sigmoid(X[test_index].dot(w_x))
        prdict_y[prdict_y>=0.5]=1
        prdict_y[prdict_y<0.5]=0

        print(classification_report(y[test_index],prdict_y))
        
        rec.append(recall_score(y[test_index],prdict_y))
        G_m.append(np.sqrt(recall_score(y[test_index],prdict_y)*recall_score(y[test_index],prdict_y,pos_label=0)))
        pre.append(precision_score(y[test_index],prdict_y))
        bacc.append(balanced_accuracy_score(y[test_index],prdict_y))
        fbeta.append(fbeta_score(y[test_index],prdict_y,beta=max(2,np.log(y_sample.shape[0]/sum(y_sample)-1))))
        T.append(end-start)
   



    return G_m,bacc,fbeta,rec,pre,T

In [30]:
result_XX = fivetrials_XX(X_sample_XX,y_sample)

G_mean=result_XX[0]
Bacc=result_XX[1]
# summarize performance
recall=result_XX[3]
Fbeta=result_XX[2]
T=result_XX[-1]
acc_s=np.mean(np.array(result_XX)[:4,:],axis=0)
	# summarize and store
print('>%s: Average G-mean:%.3f(%.3f) ' % ('max_mean loss:',np.mean(G_mean),np.std(G_mean)))
print('>%s: Average Balanced_Acc: %.3f(%.3f) ' % ('max_mean loss:',np.mean(Bacc),np.std(Bacc)))
print('>%s: Average Fbeta: %.3f(%.3f)' % ('max_mean loss:',np.mean(Fbeta),np.std(Fbeta)))
print('>%s: Average Recall: %.3f(%.3f)' % ('max_mean loss:',np.mean(recall),np.std(recall)))    
print('>%s: Average Training time: %.3f(%.3f)' % ('max_mean loss:',np.mean(T),np.std(T)))
print('>%s: Average accuracy_score: %.3f(%.3f)' % ('max_mean loss:',np.mean(acc_s),np.std(acc_s)))
print('>%s: Average Score: %.3f(%.3f)' % ('max_mean loss:',np.mean(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1)),np.std(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1))))
          


              precision    recall  f1-score   support

           0       0.96      0.89      0.92        62
           1       0.36      0.67      0.47         6

    accuracy                           0.87        68
   macro avg       0.66      0.78      0.70        68
weighted avg       0.91      0.87      0.88        68

              precision    recall  f1-score   support

           0       0.98      0.89      0.93        62
           1       0.36      0.80      0.50         5

    accuracy                           0.88        67
   macro avg       0.67      0.84      0.72        67
weighted avg       0.94      0.88      0.90        67

              precision    recall  f1-score   support

           0       0.98      0.84      0.90        61
           1       0.33      0.83      0.48         6

    accuracy                           0.84        67
   macro avg       0.66      0.83      0.69        67
weighted avg       0.92      0.84      0.86        67

              preci

# Mean-uncertain method

In [19]:
import testjx

'''Upper prediction'''

def sigmoid_(x,u):
    return 1.0/(1.0+np.exp(-x-u))
'''max-mean method to estimate mean-uncertainty of x'''
def meanuncertainty(x,n):
    r=[]
    for i in range(0,len(x)+1-n,n//10):
        r.append(np.mean(x[i:i+n]))
    return min(r),max(r)

In [20]:
'''Return the max-mean training error for LR'''
def equa_m(x,n,r):
    pre_in=(np.dot(x_tr,model_in.coef_.T)+model_in.intercept_).reshape(x_tr.shape[0],)
    
    ini_err=y_tr-sigmoid_(pre_in,x)##ini_err is the predicted err of training set, based on LR

    for k in range(ini_err.shape[0]):
        if y_tr[k]==1:
            ini_err[k]=r*ini_err[k]

    return meanuncertainty(ini_err,n)[1]


In [21]:
'''Return the max-mean training error for a given model and dataset'''
def equa_fivem(x,n,r,Mod,x_train,y_train):
    pre_in=(np.dot(x_train,Mod.coef_.T)+Mod.intercept_).reshape(x_train.shape[0],)
    
    ini_err=y_train-sigmoid_(pre_in,x)##ini_err is the predicted err of training set, based on LR

    for k in range(ini_err.shape[0]):
        if y_train[k]==1:
            ini_err[k]=r*ini_err[k]

    
    return testjx.testjx.meanuncertainty(ini_err.tolist(),n)[1]


In [22]:
'''Parameter selection:Average performance on 5 cvsets'''
metrics_names=['Balanced_acc','F2_score','Recall','Precision']
def evaluate_model(X,y,m,model=LogisticRegression()):
    
    bacc,f2,rec,pre=list(),list(),list(),list()
    for train_index,test_index in cv_.split(X,y):
#                 X_resampled_smote,y_resampled_smote=SMOTE().fit_resample(X[train_index], y[train_index])
                model.fit(X[train_index],y[train_index])
                prob_y=sigmoid_((np.dot(X[test_index],model.coef_.T)+model.intercept_),m)
                
                
                prdict_y=np.round(prob_y)
                rec.append(recall_score(y[test_index],prdict_y))
                pre.append(precision_score(y[test_index],prdict_y))
                
                bacc.append(balanced_accuracy_score(y[test_index],prdict_y))
                f2.append(fbeta_score(y[test_index],prdict_y,beta=2))
 
    return bacc,f2,rec,pre

In [25]:
'''mean-uncertainty'''
mean_lr=np.array([fsolve(lambda x:equa_m(x,n,0.5*y_sample.shape[0]/sum(y_sample)),0.5 ) for n in range(50,200,10)])


In [52]:
'''mean-uncertainty'''
mean_lr=np.array([fsolve(lambda x:equa_m(x,n,0.5*11.58),0.5 ) for n in range(50,168,5)])


In [42]:
'''ON each (train,test), we train LR first,then select the optimal parameter and estimate the upper mean, last, 
we obtain the upper predictioin and report its performances.
'''

metrics_names=['Balanced_acc','F2_score','Recall','Precision']
def Fivetrails_mean(k,X,y,model=LogisticRegression()):
    
    G_m,bacc,f2,rec,pre,T=list(),list(),list(),list(),list(),list()
    for train_index,test_index in cv.split(X,y):
                s_1=time.time()
                model.fit(X[train_index],y[train_index])
                t_1=time.time()
                all_results = []
                c=0
                
                mean_u=np.array([fsolve(lambda x:equa_fivem(x,n,0.5*(k+1),model,X[train_index],y[train_index]),0.5 ) for n in range(50,200,10)])

                for i in mean_u.reshape(-1):
                    result=np.mean(evaluate_model(X[train_index],y[train_index],i),axis=1)
                    metric_res = {'window':10+10*c,'upper_mean': i}
                    c+=1
                    for name, value in zip(metrics_names, result):
                            metric_res[name] = value


                    all_results.append(metric_res)
                eva=pd.DataFrame(all_results)
#                 eva.to_csv('{:}: datapre_diabetesCV_which_mu_to_set_inLRBIAS.csv'.format(k))
                bias=eva[eva.iloc[:,-4]==(eva.iloc[:,-4]).max()]#Select the otimal parameter with best B-acc
                
                print("optimal upper_mean by CV: %.3f "%(bias['upper_mean'].mean()))
                prob_y=sigmoid_((np.dot(X[test_index],model.coef_.T)+model.intercept_).reshape(X[test_index].shape[0],),bias['upper_mean'].mean()
                             )
                
                prdict_y=np.round(prob_y)
                print(classification_report(y[test_index],prdict_y))
                rec.append(recall_score(y[test_index],prdict_y))
                pre.append(precision_score(y[test_index],prdict_y))
                G_m.append(np.sqrt(recall_score(y[test_index],prdict_y)*recall_score(y[test_index],prdict_y,pos_label=0)))
                bacc.append(balanced_accuracy_score(y[test_index],prdict_y))
                f2.append(fbeta_score(y[test_index],prdict_y,beta=max(2,np.log(k))))
                T.append(t_1-s_1)
    return G_m,bacc,f2,rec,pre,T


In [43]:
result = Fivetrails_mean(y_sample.shape[0]/sum(y_sample)-1,X_sample,y_sample)
# 
G_mean=result[0]
Bacc=result[1]
# summarize performance
recall=result[3]
Fbeta=result[2]
T=result[-1]
acc_s=np.mean(np.array(result)[:4,:],axis=0)

print('>%s: Average G-mean:%.3f(%.3f) ' % ('LR_mean:',np.mean(G_mean),np.std(G_mean)))
print('>%s: Average Balanced_Acc: %.3f(%.3f) ' % ('LR_mean:',np.mean(Bacc),np.std(Bacc)))
print('>%s: Average Fbeta: %.3f(%.3f)' % ('LR_mean:',np.mean(Fbeta),np.std(Fbeta)))
print('>%s: Average Recall: %.3f(%.3f)' % ('LR_mean:',np.mean(recall),np.std(recall)))    
print('>%s: Average Training time: %.3f(%.3f)' % ('LR_mean:',np.mean(T),np.std(T)))
print('>%s: Average Score: %.3f(%.3f)' % ('LR_mean:',np.mean(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1)),np.std(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1))))

# print('>%s: Average accuracy_score: %.3f(%.3f)' % ('IR='+str(k),np.mean(acc_s),np.std(acc_s)))

optimal upper_mean by CV: 1.754 
              precision    recall  f1-score   support

           0       0.97      0.97      0.97        62
           1       0.67      0.67      0.67         6

    accuracy                           0.94        68
   macro avg       0.82      0.82      0.82        68
weighted avg       0.94      0.94      0.94        68

optimal upper_mean by CV: 1.802 
              precision    recall  f1-score   support

           0       0.98      0.90      0.94        62
           1       0.40      0.80      0.53         5

    accuracy                           0.90        67
   macro avg       0.69      0.85      0.74        67
weighted avg       0.94      0.90      0.91        67

optimal upper_mean by CV: 2.032 
              precision    recall  f1-score   support

           0       0.98      0.85      0.91        61
           1       0.36      0.83      0.50         6

    accuracy                           0.85        67
   macro avg       0.67      

# VOlatility-uncertain method

In [62]:
'''Define the upper and lower predictions'''

def F_u(x,arr):
    p=[]
    for c in arr:
        
        if c <0:
            p.append(2*x[1]*st.norm.cdf(c/x[1])/(x[0]+x[1]))
        else:
            p.append(1-2*x[0]*st.norm.cdf(-c/x[0])/(x[0]+x[1]))
    return np.array(p)
def F_L(x,arr):
    p=[]
    for c in arr:
        
        if c <0:
            p.append(2*x[0]*st.norm.cdf(c/x[0])/(x[0]+x[1]))
        else:
            p.append(1-2*x[1]*st.norm.cdf(-c/x[1])/(x[0]+x[1]))
    return np.array(p)

In [63]:
'''Return the volatility-uncetainty of LR-error'''

def equa_v(x,n,r):
 
    pre_in=(np.dot(x_tr,model_in.coef_.T)+model_in.intercept_).reshape(x_tr.shape[0],)
    err_u=y_tr-F_u(x,pre_in)
    err_L=y_tr-F_L(x,pre_in)
    for k in range(y_tr.shape[0]):
    
        if y_tr[k]==1:
            err_u[k]=r*err_u[k]
            err_L[k]=r*err_L[k]
    
    return np.array([testjx.testjx.meanuncertainty(err_u,n)[1],testjx.testjx.meanuncertainty(err_L,n)[0]])

In [64]:
'''Volatility-uncertainty'''
var_lr=np.array([fsolve(lambda x:equa_v(x,n,0.5*y_sample.shape[0]/sum(y_sample)),[0.5,1.5] ) for n in range(50,168,5)])


In [65]:
'''Parameter selection: average performances on five cv sets'''


def var_evaluate_model(X,y,m,model=LogisticRegression()):
    
    bacc,f2,rec,pre=list(),list(),list(),list()
   
    for train_index,test_index in cv_.split(X,y):
#                 X_resampled_smote,y_resampled_smote=SMOTE().fit_resample(X[train_index], y[train_index])
                model.fit(X[train_index],y[train_index])
                prob_y=F_u(m,(np.dot(X[test_index],model.coef_.T)+model.intercept_).reshape(X[test_index].shape[0],))
        
#                 auc.append(roc_auc_score(y[test_index],prob_y))
                
                prdict_y=np.round(prob_y)
                rec.append(recall_score(y[test_index],prdict_y))
                pre.append(precision_score(y[test_index],prdict_y))
                
                bacc.append(balanced_accuracy_score(y[test_index],prdict_y))
                f2.append(fbeta_score(y[test_index],prdict_y,beta=2))
 
    return bacc,f2,rec,pre

In [66]:
'''ON each (train,test), we train LR first,then select the optimal parameter and estimate the upper and lower volatilities, last, 
we obtain the upper predictioin and report its performances.
'''

def Fivetrails_var(X,y,model=LogisticRegression()):
    
    G_m,bacc,f2,rec,pre,T=list(),list(),list(),list(),list(),list()
    for train_index,test_index in cv.split(X,y):
                s_1=time.time()
                model.fit(X[train_index],y[train_index])
                t_1=time.time()
                all_results = []
                c=0
         
                for i in var_lr:
                    result=np.mean(var_evaluate_model(X[train_index],y[train_index],i),axis=1)
                    metric_res = {'window':10+5*c,'upper_mean': i}
                    c+=1
                    for name, value in zip(metrics_names, result):
                            metric_res[name] = value


                    all_results.append(metric_res)
                eva=pd.DataFrame(all_results)
#                 eva.to_csv('{:}: datapre_diabetesCV_which_mu_to_set_inLRBIAS.csv'.format(k))
                bias=eva[eva.iloc[:,-4]==(eva.iloc[:,-4]).max()]
                print("optimal volatility by CV:",np.array(bias['upper_mean']))
                end=time.time()
                
                prob_y=F_u(np.array(bias['upper_mean'])[0],(np.dot(X[test_index],model.coef_.T)+model.intercept_).reshape(X[test_index].shape[0],))
                
                
                prdict_y=np.round(prob_y)
                print(classification_report(y[test_index],prdict_y))
                rec.append(recall_score(y[test_index],prdict_y))
                pre.append(precision_score(y[test_index],prdict_y))
                G_m.append(np.sqrt(recall_score(y[test_index],prdict_y)*recall_score(y[test_index],prdict_y,pos_label=0)))
                bacc.append(balanced_accuracy_score(y[test_index],prdict_y))
                f2.append(fbeta_score(y[test_index],prdict_y,beta=max(2,np.log(y_sample.shape[0]/sum(y_sample)-1))))
                T.append(t_1-s_1)
    return G_m,bacc,f2,rec,pre,T


In [69]:
result_v = Fivetrails_var(X_sample,y_sample)
G_mean=result_v[0]
Bacc=result_v[1]
# summarize performance
recall=result_v[3]
Fbeta=result_v[2]
T=result_v[-1]
acc_s=np.mean(np.array(result)[:4,:],axis=0)

print('>%s: Average G-mean:%.3f(%.3f) ' % ('LR_volatility:',np.mean(G_mean),np.std(G_mean)))
print('>%s: Average Balanced_Acc: %.3f(%.3f) ' % ('LR_volatility:',np.mean(Bacc),np.std(Bacc)))
print('>%s: Average Fbeta: %.3f(%.3f)' % ('LR_volatility:',np.mean(Fbeta),np.std(Fbeta)))
print('>%s: Average Recall: %.3f(%.3f)' % ('LR_volatility:',np.mean(recall),np.std(recall)))    
print('>%s: Average Training time: %.3f(%.3f)' % ('LR_volatility:',np.mean(T),np.std(T)))
print('>%s: Average Score: %.3f(%.3f)' % ('LR_volatility:',np.mean(0.99*acc_s+0.01/(np.array(result_v)[-1,:]/10*9+1)),np.std(0.99*acc_s+0.01/(np.array(result_v)[-1,:]/10*9+1))))


optimal volatility by CV: [array([5.15812233, 7.97666962])]
              precision    recall  f1-score   support

           0       0.97      0.95      0.96        62
           1       0.57      0.67      0.62         6

    accuracy                           0.93        68
   macro avg       0.77      0.81      0.79        68
weighted avg       0.93      0.93      0.93        68

optimal volatility by CV: [array([5.31621807, 7.89987979])]
              precision    recall  f1-score   support

           0       0.97      0.95      0.96        62
           1       0.50      0.60      0.55         5

    accuracy                           0.93        67
   macro avg       0.73      0.78      0.75        67
weighted avg       0.93      0.93      0.93        67

optimal volatility by CV: [array([3.59021476, 6.86765524])]
              precision    recall  f1-score   support

           0       0.98      0.85      0.91        61
           1       0.36      0.83      0.50         6

  